In [ ]:
#neulab/codebert-cpp
#microsoft/graphcodebert-base
#

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm.notebook import tqdm

In [19]:
a = AutoTokenizer.from_pretrained('OpenMatch/cocodr-base-msmarco')
a.encode_plus('hi hi hi','he he he', 
 add_special_tokens=True,
            max_length=16,
            truncation=True,
            return_tensors="pt",
            padding='max_length')

{'input_ids': tensor([[ 101, 7632, 7632, 7632,  102, 2002, 2002, 2002,  102,    0,    0,    0,
            0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

In [16]:
model = AutoModel.from_pretrained('OpenMatch/cocodr-base-msmarco')
model.to('cpu')

config.json:   0%|          | 0.00/977 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [2]:
class NewsDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.code1 = data['code1'].to_numpy()
        self.code2 = data['code2'].to_numpy()
        if 'similar' in data.columns:
            self.labels = data['similar'].to_numpy()
        else:
            self.labels = None

    def __len__(self):
        return len(self.code1)

    def __getitem__(self, idx):
        code1 = self.code1[idx]
        code2 = self.code2[idx]
        
        encoding = self.tokenizer(
            code1,
            code2,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_tensors="pt",
            padding='max_length'
        )


#         encoding_first = self.tokenizer(
#             code1,
#             add_special_tokens=True,
#             max_length=self.max_len//2,
#             truncation=True,
#             return_tensors="pt"
#         )
#         encoding_second = self.tokenizer(
#             code2,
#             add_special_tokens=True,
#             max_length=self.max_len//2,
#             padding='max_length',
#             truncation=True,
#             return_tensors="pt"
#         )
#         ids = torch.concat([encoding_first['input_ids'][0],
#               encoding_second['input_ids'][0][1:],
#               torch.ones(self.max_len+1-len(encoding_first['input_ids'][0]) - len(encoding_second['input_ids'][0]),dtype=torch.int)])
#         mask = torch.concat([encoding_first['attention_mask'][0],
#               encoding_second['attention_mask'][0][1:],
#              torch.zeros(self.max_len+1-len(encoding_first['attention_mask'][0]) - len(encoding_second['attention_mask'][0]),dtype=torch.int)])       
#         if self.labels is not None:
#             label = self.labels[idx]
#             return {'input_ids': ids,
#                     'attention_mask': mask,
#                     'labels': torch.tensor(label, dtype=torch.long)}
#         else:
#             return {'input_ids': ids,
#                     'attention_mask': mask
#                    }
        

        if self.labels is not None:
            label = self.labels[idx]
            return {'input_ids': encoding['input_ids'][0],
                    'attention_mask': encoding['attention_mask'][0],
                    'labels': torch.tensor(label, dtype=torch.long)}
        else:
            return {'input_ids': encoding['input_ids'][0],
                    'attention_mask': encoding['attention_mask'][0]
                   }


In [3]:
#model_name = "neulab/codebert-cpp"
model_name = 'microsoft/graphcodebert-base'
train_data = pd.read_csv('./sample_train.csv')
test_data = pd.read_csv('./test.csv')

tokenizer = AutoTokenizer.from_pretrained(model_name)
train_dataset = NewsDataset(train_data, tokenizer, max_len=512)
test_dataset = NewsDataset(test_data, tokenizer, max_len=512)

train_loader = DataLoader(train_dataset, batch_size=11, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=11, shuffle=False)

model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

/home/server4/anaconda3/envs/jy/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
for epoch in tqdm(range(100)):
    model.train()
    train_loss = 0
    acc = 0
    cnt = 0
    print(f'\nepoch : {epoch+1}')
    for i, batch in tqdm(enumerate(train_loader),leave=False,total=len(train_loader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        acc += torch.sum(labels==preds)
        cnt += torch.sum(preds)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    model.save_pretrained(f"model_codereview/cpp-{epoch}", from_pt=True)
    print(f'train_loss : {train_loss}\nacc = {acc / len(train_data)}\ncount = {cnt}')

  0%|          | 0/100 [00:00<?, ?it/s]


epoch : 1


  0%|          | 0/1819 [00:00<?, ?it/s]

train_loss : 1048.7195901200175
acc = 0.6506999731063843
count = 9364

epoch : 2


  0%|          | 0/1819 [00:00<?, ?it/s]

...

train_loss : 632.6920202057809
acc = 0.8047999739646912
count = 9814

epoch : 4


  0%|          | 0/1819 [00:00<?, ?it/s]

train_loss : 562.7559965290129
acc = 0.8299999833106995
count = 9900

epoch : 5


  0%|          | 0/1819 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
model_name = "microsoft/graphcodebert-base"

test_data = pd.read_csv('./test.csv')

tokenizer = AutoTokenizer.from_pretrained(model_name)
test_dataset = NewsDataset(test_data, tokenizer, max_len=512)

test_loader = DataLoader(test_dataset, batch_size=220, shuffle=False, num_workers=4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
model = AutoModelForSequenceClassification.from_pretrained("model_codereview/cpp-3")
model.to(device)
model.eval()
with torch.no_grad():
    test_preds = []
    for i, batch in tqdm(enumerate(test_loader),total=len(test_loader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        test_preds.extend(preds.cpu().numpy())
submission = pd.DataFrame({'pair_id': test_data['pair_id'], 'similar': test_preds})
submission.to_csv('./submission.csv', index=False)

  0%|          | 0/2705 [00:00<?, ?it/s]

In [5]:
sum(pd.read_csv('submission.csv')['similar'])

208212